# Data Merge

This notebook outlines the steps taken to merge the required suburb attributes into a single dataset.

It then shows the steps taken in creating and populating the sqlalchemy database which is to be used in the app.

In [1]:
import os
import numpy as np
import pandas as pd

import helper

# make all output interactive
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# Define universal constants
# ================================
DATA_PATH = os.path.join(os.getcwd(), "../data/")
RAW_DATA_PATH = os.path.join(DATA_PATH, "raw")
DERIVED_DATA_PATH = os.path.join(DATA_PATH, "derived")
CORR_DATA_PATH = os.path.join(DATA_PATH, "correspondence")

## Processing

Each 'suburb' in the dataset, and hence the application, will be based on the Suburb and Locality non-ABS structure.


In [3]:
# Define merge-section-specific constants
# ================================

# 2016 census data
DATA_2016_CENSUS_DATA_PATH = os.path.join(RAW_DATA_PATH, "2016_GCP_SSC_for_VIC_short-header/2016 Census GCP State Suburbs for VIC")
DATA_2016_CENSUS_METADATA_PATH = os.path.join(RAW_DATA_PATH, "2016_GCP_SSC_for_VIC_short-header/Metadata")
DATA_2016_CENSUS_DATA_FILENAMES = os.listdir(DATA_2016_CENSUS_DATA_PATH)

# 2021 census data
DATA_2021_CENSUS_DATA_PATH = os.path.join(RAW_DATA_PATH, "2021_GCP_SAL_for_VIC_short-header/2021 Census GCP Suburbs and Localities for VIC")
DATA_2021_CENSUS_METADATA_PATH = os.path.join(RAW_DATA_PATH, "2021_GCP_SAL_for_VIC_short-header/Metadata")
DATA_2021_CENSUS_DATA_FILENAMES = os.listdir(DATA_2021_CENSUS_DATA_PATH)

# 2021 crime statistics data
DATA_2021_CRIME_DATA_PATH = os.path.join(RAW_DATA_PATH, "Data_Tables_LGA_Criminal_Incidents_Year_Ending_December_2021.xlsx")

# Conversion between 2016 SSC codes and 2021 SAL codes
DATA_CONVERT_SSC_SAL_PATH = os.path.join(RAW_DATA_PATH, "convert-SAL-2016-2021.csv")

# All meta data about every suburb/locality
LOCALITIES_DETAILS_PATH = os.path.join(RAW_DATA_PATH, "australian_postcodes.csv")


### Prepare census dataset (2021 only for now)

In [4]:
# All localities meta data
# ========================

suburb_metadata_df = helper.getSuburbsMetadata()
suburb_metadata_df

,postcode,locality,coordinates,lgaregion,sa1_code_2016,sa1_code_2021,sa2_code_2016,sa2_name_2016,sa2_code_2021,sa2_name_2021,sa3_code_2016,sa3_name_2016,sa3_code_2021,sa3_name_2021,sa4_code_2016,sa4_name_2016,sa4_code_2021,sa4_name_2021
0,3000,Melbourne,"(-37.8152065, 144.963937)",Melbourne,2.060411e+10,20604150327,206041122.0,Melbourne,206041503,Melbourne CBD - East,20604.0,Melbourne City,20604,Melbourne City,206.0,Melbourne - Inner,206,Melbourne - Inner
1,3002,East Melbourne,"(-37.8161444, 144.9804594)",Yarra,2.060411e+10,20604111914,206041119.0,East Melbourne,206041119,East Melbourne,20604.0,Melbourne City,20604,Melbourne City,206.0,Melbourne - Inner,206,Melbourne - Inner
2,3003,West Melbourne,"(-37.8114504, 144.9253974)",Melbourne,2.060411e+10,20604112701,206041127.0,West Melbourne,206041127,West Melbourne - Industrial,20604.0,Melbourne City,20604,Melbourne City,206.0,Melbourne - Inner,206,Melbourne - Inner
3,3004,St Kilda Road Central,"(-37.8367638, 144.9756445)",Yarra,2.060411e+10,20604112506,206041125.0,South Yarra - West,206041125,South Yarra - West,20604.0,Melbourne City,20604,Melbourne City,206.0,Melbourne - Inner,206,Melbourne - Inner
4,3004,St Kilda Road Melbourne,"(-37.8367638, 144.9756445)",Yarra,2.060411e+10,20604112506,206041125.0,South Yarra - West,206041125,South Yarra - West,20604.0,Melbourne City,20604,Melbourne City,206.0,Melbourne - Inner,206,Melbourne - Inner
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3263,3988,Poowong North,"(-38.2808915, 145.7825971)",South Gippsland,2.050311e+10,20503108916,205031089.0,Korumburra,205031089,Korumburra,20503.0,Gippsland - South West,20503,Gippsland - South West,205.0,Latrobe - Gippsland,205,Latrobe - Gippsland
3264,3960,Tidal River,"(-39.02979215080867, 146.3208933505564)",Wellington,2.050311e+10,20503109201,205031092.0,Wilsons Promontory,205031092,Wilsons Promontory,20503.0,Gippsland - South West,20503,Gippsland - South West,205.0,Latrobe - Gippsland,205,Latrobe - Gippsland
3265,3960,Wilsons Promontory,"(-38.9572966, 146.28311)",Wellington,2.050311e+10,20503109201,205031092.0,Wilsons Promontory,205031092,Wilsons Promontory,20503.0,Gippsland - South West,20503,Gippsland - South West,205.0,Latrobe - Gippsland,205,Latrobe - Gippsland
3266,3707,Bringenbrong,"(-36.1428573, 148.0518011)",Towong,1.130213e+10,11302126013,113021260.0,Tumbarumba,113021260,Tumbarumba,11302.0,Tumut - Tumbarumba,11302,Tumut - Tumbarumba,113.0,Riverina,113,Riverina


In [5]:
# 2021 census meta data
# =====================

# read file
data_2021_census_metadata_df = pd.read_excel(
    os.path.join(DATA_2021_CENSUS_METADATA_PATH, "2021Census_geog_desc_1st_release.xlsx"),
    sheet_name="2021_ASGS_Non_ABS_Structures"
)
# filter for victorian regions
data_2021_census_metadata_df = data_2021_census_metadata_df[data_2021_census_metadata_df["Census_Code_2021"].str.contains("^SAL2")]
# split lga and state component from name column where multiple places have the same name
data_2021_census_metadata_df["Census_Name_2021"] = data_2021_census_metadata_df["Census_Name_2021"].str.replace("\s*\(Vic.\)$", "", regex=True)
data_2021_census_metadata_df["Census_Name_2021"] = data_2021_census_metadata_df["Census_Name_2021"].str.replace("^(.*?)\s*\((.*?) - Vic.\)$", lambda m: f"{m.group(1)}, {m.group(2)}", regex=True)
data_2021_census_metadata_df[["locality", "lgaregion"]] = data_2021_census_metadata_df["Census_Name_2021"].str.split(", ", n=1, expand=True)
# split dataset horizontally between those with lga data and those without
data_2021_census_metadata_df_lga = data_2021_census_metadata_df[data_2021_census_metadata_df["lgaregion"].notnull()]
data_2021_census_metadata_df_nolga = data_2021_census_metadata_df[data_2021_census_metadata_df["lgaregion"].isnull()]

# TODO: Handle when the name is duplicate and the LGA region is specified
data_2021_census_metadata_df_nolga

,ASGS_Structure,Census_Code_2021,AGSS_Code_2021,Census_Name_2021,Area sqkm,locality,lgaregion
8373,SAL,SAL20001,20001,Abbeyard,327.5008,Abbeyard,None
8374,SAL,SAL20002,20002,Abbotsford,1.7405,Abbotsford,None
8375,SAL,SAL20003,20003,Aberfeldie,1.5515,Aberfeldie,None
8376,SAL,SAL20004,20004,Aberfeldy,10.8319,Aberfeldy,None
8377,SAL,SAL20005,20005,Acheron,72.6602,Acheron,None
...,...,...,...,...,...,...,...
11314,SAL,SAL22942,22942,Yuulong,52.1657,Yuulong,None
11315,SAL,SAL22943,22943,Zeerust,18.0294,Zeerust,None
11316,SAL,SAL22944,22944,Zumsteins,256.5582,Zumsteins,None
11317,SAL,SAL29494,29494,No usual address,0.0000,No usual address,None


In [6]:
# Combine all locality details and 2021 census meta data

suburb_metadata_df = suburb_metadata_df.merge(
    data_2021_census_metadata_df_nolga,
    how="right",  # only keep the rows which have data in the 2021 census
    on="locality",
    suffixes=[None, "_census"]
)
suburb_metadata_df

,postcode,locality,coordinates,lgaregion,sa1_code_2016,sa1_code_2021,sa2_code_2016,sa2_name_2016,sa2_code_2021,sa2_name_2021,...,sa4_code_2016,sa4_name_2016,sa4_code_2021,sa4_name_2021,ASGS_Structure,Census_Code_2021,AGSS_Code_2021,Census_Name_2021,Area sqkm,lgaregion_census
0,3737.0,Abbeyard,"(-36.986557, 146.7708948)",Wellington,2.040311e+10,2.040311e+10,204031069.0,Bright - Mount Beauty,204031069.0,Bright - Mount Beauty,...,204.0,Hume,204.0,Hume,SAL,SAL20001,20001,Abbeyard,327.5008,None
1,3067.0,Abbotsford,"(-37.8023601, 144.9983623)",Yarra,2.060711e+10,2.060711e+10,206071139.0,Abbotsford,206071139.0,Abbotsford,...,206.0,Melbourne - Inner,206.0,Melbourne - Inner,SAL,SAL20002,20002,Abbotsford,1.7405,None
2,3040.0,Aberfeldie,"(-37.7603346, 144.8956625)",Moreland,2.060311e+10,2.060315e+10,206031114.0,Essendon - Aberfeldie,206031501.0,Essendon (West) - Aberfeldie,...,206.0,Melbourne - Inner,206.0,Melbourne - Inner,SAL,SAL20003,20003,Aberfeldie,1.5515,None
3,3825.0,Aberfeldy,"(-37.6979736, 146.3609099)",Wellington,2.050111e+10,2.050111e+10,205011077.0,Mount Baw Baw Region,205011077.0,Mount Baw Baw Region,...,205.0,Latrobe - Gippsland,205.0,Latrobe - Gippsland,SAL,SAL20004,20004,Aberfeldy,10.8319,None
4,3714.0,Acheron,"(-37.2587906, 145.702904)",Murrindindi,2.040111e+10,2.040111e+10,204011054.0,Alexandra,204011054.0,Alexandra,...,204.0,Hume,204.0,Hume,SAL,SAL20005,20005,Acheron,72.6602,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2911,3237.0,Yuulong,"(-38.7357943, 143.3031007)",Corangamite,2.170315e+10,2.170315e+10,217031476.0,Otway,217031476.0,Otway,...,217.0,Warrnambool and South West,217.0,Warrnambool and South West,SAL,SAL22942,22942,Yuulong,52.1657,None
2912,3634.0,Zeerust,"(-36.2763544, 145.404537)",Moira,2.160314e+10,2.160314e+10,216031418.0,Shepparton Region - East,216031418.0,Shepparton Surrounds - East,...,216.0,Shepparton,216.0,Shepparton,SAL,SAL22943,22943,Zeerust,18.0294,None
2913,3401.0,Zumsteins,"(-37.0932892, 142.3855776)",Yarriambiack,2.150114e+10,2.150114e+10,215011392.0,Stawell,215011392.0,Stawell,...,215.0,North West,215.0,North West,SAL,SAL22944,22944,Zumsteins,256.5582,None
2914,NaN,No usual address,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,SAL,SAL29494,29494,No usual address,0.0000,None


### Prepare crime dataset

In [13]:
# read file
crime_df = pd.read_excel(DATA_2021_CRIME_DATA_PATH, sheet_name="Table 03")


In [16]:
# filter for 2021 statistics
crime_df = crime_df[crime_df["Year"] == 2021]


In [18]:
# drop unnecessary columns
crime_df = crime_df.drop(["Year", "Year ending", "Postcode", "Unnamed: 9"], axis=1)


In [20]:
# group by locality and offence division and pivot table
crime_df = crime_df.groupby(["Suburb/Town Name", "Offence Division"], ).sum().unstack()


In [22]:
# clean up groupby result
crime_df = crime_df.reset_index()
crime_df.columns = ["locality", "A Crimes against the person", "B Property and deception offences",
                "C Drug offences", "D Public order and security offences", "E Justice procedures offences", "F Other offences"]


In [23]:
# remove null values with 0
crime_df.fillna(0, inplace=True)

crime_df

,locality,A Crimes against the person,B Property and deception offences,C Drug offences,D Public order and security offences,E Justice procedures offences,F Other offences
0,Abbeyard,0.0,1.0,0.0,0.0,0.0,0.0
1,Abbotsford,149.0,758.0,61.0,56.0,94.0,21.0
2,Aberfeldie,14.0,88.0,3.0,2.0,15.0,0.0
3,Acheron,0.0,2.0,0.0,0.0,0.0,0.0
4,Adams Estate,2.0,0.0,0.0,0.0,2.0,0.0
...,...,...,...,...,...,...,...
2314,Youanmite,0.0,1.0,0.0,0.0,0.0,0.0
2315,Youarang,1.0,1.0,1.0,0.0,0.0,0.0
2316,Yulecart,0.0,0.0,0.0,1.0,1.0,0.0
2317,Yuroke,1.0,5.0,0.0,2.0,1.0,0.0


### Prepare environment and industry data

In [24]:
# Environment dataset
# ===================

env_df = pd.read_excel(
    os.path.join(RAW_DATA_PATH, "2011-2020 ABS Regional Data/LandEnvironment.xlsx"),
    sheet_name="Table 1",
    header=6,
    skipfooter=6
)
env_df = env_df[env_df["Code"].astype(str).str.contains("^2[0-9]{8}")]
env_df = env_df[env_df["Year"] == 2020]
env_df = env_df.replace("-", np.nan)
env_df = env_df.fillna(0)

env_df


,Code,Label,Year,Land area (ha),Area of agricultural land (ha),Area irrigated (ha),Volume of water applied (ML),Application rate - total area watered and used (ML/ha),Indigenous protected land areas (no.),National parks (no.),...,Other perpetual lease (ha),Pastoral term lease (ha),Other term lease (ha),Other lease (ha),Nature conservation reserve (ha),Multiple-use public forest (ha),Other Crown purposes (ha),Other Crown land (ha),No data (ha).1,Total (ha).1
7223,201011001,Alfredton,2020,5271.1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7230,201011002,Ballarat,2020,1237.9,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7237,201011003,Ballarat - North,2020,9235.8,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7244,201011004,Ballarat - South,2020,3285.4,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7251,201011005,Buninyong,2020,5158.6,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10422,217031476,Otway,2020,150589.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10429,217041477,Moyne - East,2020,327579.9,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10436,217041478,Moyne - West,2020,203776.3,0.0,0.0,0.0,0.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10443,217041479,Warrnambool - North,2020,6242.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
# Industry dataset
# ===================

ind_df = pd.read_excel(
    os.path.join(RAW_DATA_PATH, "2011-2020 ABS Regional Data/EconomyIndustry.xlsx"),
    sheet_name="Table 1",
    header=6,
    skipfooter=6
)
ind_df = ind_df[ind_df["Code"].astype(str).str.contains("^2[0-9]{8}")]
ind_df = ind_df[ind_df["Year"] == 2020]
ind_df = ind_df.replace("-", np.nan)
ind_df = ind_df.fillna(0)

ind_df


,Code,Label,Year,Number of non-employing businesses,Number of employing businesses: 1-4 employees,Number of employing businesses: 5-19 employees,Number of employing businesses: 20 or more employees,Total number of businesses,Number of non employing business entries,Number of employing business entries: 1-4 employees,...,"Rental, hiring and real estate services (%)","Professional, scientific and technical services (%)",Administrative and support services (%),Public administration and safety (%),Education and training (%),Health care and social assistance (%),Arts and recreation services (%),Other services (%),Industry of employment - inadequately described or not stated (%),Total persons employed (no.)
7223,201011001,Alfredton,2020,579.0,216.0,92.0,25.0,914.0,102.0,31.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7230,201011002,Ballarat,2020,1945.0,645.0,359.0,89.0,3036.0,237.0,71.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7237,201011003,Ballarat - North,2020,978.0,488.0,141.0,34.0,1638.0,143.0,54.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7244,201011004,Ballarat - South,2020,733.0,330.0,104.0,27.0,1186.0,125.0,59.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7251,201011005,Buninyong,2020,328.0,117.0,25.0,8.0,469.0,36.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10422,217031476,Otway,2020,321.0,132.0,44.0,15.0,507.0,33.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10429,217041477,Moyne - East,2020,689.0,277.0,112.0,6.0,1090.0,51.0,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10436,217041478,Moyne - West,2020,754.0,295.0,123.0,18.0,1187.0,79.0,30.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10443,217041479,Warrnambool - North,2020,737.0,286.0,97.0,25.0,1149.0,118.0,35.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Merge datasets

- 2016 census data uses SSC
- 2021 census data uses SAL
- 2021 crime data uses suburb name
- Environment data (2020) uses SA2 (2016)
- Industry data (2020) uses SA2 (2016)

In [26]:
data = suburb_metadata_df.copy()

In [27]:
# Join data from 2021 Census
# ==========================

for file in DATA_2021_CENSUS_DATA_FILENAMES:
    file_path = os.path.join(DATA_2021_CENSUS_DATA_PATH, file)
    temp = pd.read_csv(file_path)
    data = data.merge(
        temp, 
        how="left", 
        left_on="Census_Code_2021", 
        right_on="SAL_CODE_2021",
        suffixes=[None, "_" + file.split("_")[1]]
    )


In [28]:
# Join data from crime statistics dataset
# =======================================

data = data.merge(crime_df, how="left", on="locality")


In [29]:
# Join data from environment and industry datasets
# ================================================

data = data.merge(env_df, how="left", left_on="sa2_code_2016", right_on="Code")
data = data.merge(ind_df, how="left", left_on="sa2_code_2016", right_on="Code")


In [30]:
data.head()

,postcode,locality,coordinates,lgaregion,sa1_code_2016,sa1_code_2021,sa2_code_2016,sa2_name_2016,sa2_code_2021,sa2_name_2021,...,"Rental, hiring and real estate services (%)","Professional, scientific and technical services (%)",Administrative and support services (%),Public administration and safety (%),Education and training (%),Health care and social assistance (%),Arts and recreation services (%),Other services (%),Industry of employment - inadequately described or not stated (%),Total persons employed (no.)
0,3737.0,Abbeyard,"(-36.986557, 146.7708948)",Wellington,2.040311e+10,2.040311e+10,204031069.0,Bright - Mount Beauty,204031069.0,Bright - Mount Beauty,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3067.0,Abbotsford,"(-37.8023601, 144.9983623)",Yarra,2.060711e+10,2.060711e+10,206071139.0,Abbotsford,206071139.0,Abbotsford,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3040.0,Aberfeldie,"(-37.7603346, 144.8956625)",Moreland,2.060311e+10,2.060315e+10,206031114.0,Essendon - Aberfeldie,206031501.0,Essendon (West) - Aberfeldie,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3825.0,Aberfeldy,"(-37.6979736, 146.3609099)",Wellington,2.050111e+10,2.050111e+10,205011077.0,Mount Baw Baw Region,205011077.0,Mount Baw Baw Region,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3714.0,Acheron,"(-37.2587906, 145.702904)",Murrindindi,2.040111e+10,2.040111e+10,204011054.0,Alexandra,204011054.0,Alexandra,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
data.to_csv("../data/derived/SuburbRawdata.csv", index=False)

## Feature extraction

In [15]:
new_data = suburb_metadata_df.copy()

### Accomodation attributes

In [16]:
# acc_house_onebed
new_data["acc_house_onebed"] = (data["Separate_house_NofB_1"] + data["Se_d_r_or_t_h_t_Tot_NofB_1"])/data['Tot_Tot_P']
new_data["acc_house_onebed"].fillna(0, inplace=True)
# acc_house_twobed
new_data["acc_house_twobed"] = (data["Separate_house_NofB_2"] + data["Se_d_r_or_t_h_t_Tot_NofB_2"])/data['Tot_Tot_P']
new_data["acc_house_twobed"].fillna(0, inplace=True)
# acc_house_threebed
new_data["acc_house_threebed"] = (data["Separate_house_NofB_3"] + data["Se_d_r_or_t_h_t_Tot_NofB_3"])/data['Tot_Tot_P']
new_data["acc_house_threebed"].fillna(0, inplace=True)
# acc_house_fourplusbed
new_data["acc_house_fourplusbed"] = (data["Separate_house_NofB_4"] + data["Se_d_r_or_t_h_t_Tot_NofB_4"] + data["Separate_house_NofB_5"] + data["Se_d_r_or_t_h_t_Tot_NofB_5"] + data["Separate_house_NofB_6_or_m"] + data["Se_d_r_or_t_h_t_Tot_NofB_6_m"])/data['Tot_Tot_P']
new_data["acc_house_fourplusbed"].fillna(0, inplace=True)
# acc_apartment_onebed
new_data["acc_apartment_onebed"] = data["Flt_apart_Tot_NofB_1"]/data['Tot_Tot_P']
new_data["acc_apartment_onebed"].fillna(0, inplace=True)
# acc_apartment_twobed
new_data["acc_apartment_twobed"] = data["Flt_apart_Tot_NofB_2"]/data['Tot_Tot_P']
new_data["acc_apartment_twobed"].fillna(0, inplace=True)
# acc_apartment_threebed
new_data["acc_apartment_threebed"] = data["Flt_apart_Tot_NofB_3"]/data['Tot_Tot_P']
new_data["acc_apartment_threebed"].fillna(0, inplace=True)
# acc_apartment_fourplusbed
new_data["acc_apartment_fourplusbed"] = (data["Flt_apart_Tot_NofB_4"] + data["Flt_apart_Tot_NofB_5"] + data["Flt_apart_Tot_NofB_6_or_m"])/data['Tot_Tot_P']
new_data["acc_apartment_fourplusbed"].fillna(0, inplace=True)
# acc_rented_house_relative
new_data["acc_rented_house_relative"] = (data["R_Tot_DS_Sep_house"] + data["R_Tot_DS_SemiD_ro_or_tce_h_th"]) / (data["Total_DS_Sep_house"] + data["Total_DS_SemiD_ro_or_tce_h_th"])
new_data["acc_rented_house_relative"].fillna(0, inplace=True)
# acc_rented_apartment_relative
new_data["acc_rented_apartment_relative"] = data["R_Tot_DS_Flat_apart"] / data["Total_DS_Flat_apart"]
new_data["acc_rented_apartment_relative"].fillna(0, inplace=True)
# acc_shared_relative
new_data["acc_shared_relative"] = data["Tot_Group_H"] / data["Tot_Tot_G42"]
new_data["acc_shared_relative"].fillna(0, inplace=True)
# acc_rent_1_74
new_data["acc_rent_1_74"] = data["R_1_74_Tot"]/data['Tot_Tot_P']
new_data["acc_rent_1_74"].fillna(0, inplace=True)
# acc_rent_75_99
new_data["acc_rent_75_99"] = data["R_75_99_Tot"]/data['Tot_Tot_P']
new_data["acc_rent_75_99"].fillna(0, inplace=True)
# acc_rent_100_149
new_data["acc_rent_100_149"] = data["R_100_149_Tot"]/data['Tot_Tot_P']
new_data["acc_rent_100_149"].fillna(0, inplace=True)
# acc_rent_150_199
new_data["acc_rent_150_199"] = data["R_150_199_Tot"]/data['Tot_Tot_P']
new_data["acc_rent_150_199"].fillna(0, inplace=True)
# acc_rent_200_224
new_data["acc_rent_200_224"] = data["R_200_224_Tot"]/data['Tot_Tot_P']
new_data["acc_rent_200_224"].fillna(0, inplace=True)
# acc_rent_225_274
new_data["acc_rent_225_274"] = data["R_225_274_Tot"]/data['Tot_Tot_P']
new_data["acc_rent_225_274"].fillna(0, inplace=True)
# acc_rent_275_349
new_data["acc_rent_275_349"] = data["R_275_349_Tot"]/data['Tot_Tot_P']
new_data["acc_rent_275_349"].fillna(0, inplace=True)
# acc_rent_350_449
new_data["acc_rent_350_449"] = data["R_350_449_Tot"]/data['Tot_Tot_P']
new_data["acc_rent_350_449"].fillna(0, inplace=True)
# acc_rent_450_549
new_data["acc_rent_450_549"] = data["R_450_549_Tot"]/data['Tot_Tot_P']
new_data["acc_rent_450_549"].fillna(0, inplace=True)
# acc_rent_550_649
new_data["acc_rent_550_649"] = data["R_550_649_Tot"]/data['Tot_Tot_P']
new_data["acc_rent_550_649"].fillna(0, inplace=True)
# acc_rent_650_749
new_data["acc_rent_650_749"] = data["R_650_749_Tot"]/data['Tot_Tot_P']
new_data["acc_rent_650_749"].fillna(0, inplace=True)
# acc_rent_750_849
new_data["acc_rent_750_849"] = data["R_750_849_Tot"]/data['Tot_Tot_P']
new_data["acc_rent_750_849"].fillna(0, inplace=True)
# acc_rent_850_949
new_data["acc_rent_850_949"] = data["R_850_949_Tot"]/data['Tot_Tot_P']
new_data["acc_rent_850_949"].fillna(0, inplace=True)
# acc_rent_950_plus
new_data["acc_rent_950_plus"] = data["R_950_over_Tot"]/data['Tot_Tot_P']
new_data["acc_rent_950_plus"].fillna(0, inplace=True)

### Industry & Environment attributes

In [17]:
new_data["env_retail"] = data["Retail trade (no.)"]/data['Tot_Tot_P']
new_data["env_retail"].fillna(0, inplace=True)

new_data["env_accomodation_food"] = data["Accommodation and food services (no.)"]/data['Tot_Tot_P']
new_data["env_accomodation_food"].fillna(0, inplace=True)

new_data["env_public_admin"] = data["Public administration and safety (no.)"]/data['Tot_Tot_P']
new_data["env_public_admin"].fillna(0, inplace=True)

new_data["env_healthcare_social_assist"] = data["Health care and social assistance (no.)"]/data['Tot_Tot_P']
new_data["env_healthcare_social_assist"].fillna(0, inplace=True)

new_data["env_arts_recreation"] = data["Arts and recreation services (no.)"]/data['Tot_Tot_P']
new_data["env_arts_recreation"].fillna(0, inplace=True)

new_data["env_rental_hiring_realestate"] = data["Rental, hiring and real estate services (no.)"]/data['Tot_Tot_P']
new_data["env_rental_hiring_realestate"].fillna(0, inplace=True)

new_data["env_parks"] = data["Protected land area (%)"]/data['Tot_Tot_P']
new_data["env_parks"].fillna(0, inplace=True)

### Demographics attributes

In [18]:
new_data["dem_students_relative"] = data["Tert_Tot_Tert_P"] / data["Tot_Tot_P"]
new_data["dem_students_relative"].fillna(0, inplace=True)


### Transport attributes

The code used to determine these features is located in the [1.2-TransportAttributes.ipynb](1.2-TransportAttributes.ipynb) jupyter notebook.

In [19]:
# read saved scores from disk
suburb_transport_score_df = pd.read_csv(os.path.join(DERIVED_DATA_PATH, "SuburbTransportScores.csv"))
# add new columns based on `locality` column
new_data = new_data.merge(suburb_transport_score_df)


### Safety attributes

In [20]:
# Offence division A
new_data["saf_crime_person"] = data["A Crimes against the person"] / data["Tot_Tot_P"]
new_data["saf_crime_person"].fillna(0, inplace=True)
# Offence division B
new_data["saf_crime_property"] = data["B Property and deception offences"] / data["Tot_Tot_P"]
new_data["saf_crime_property"].fillna(0, inplace=True)
# Offence division C
new_data["saf_drug_offences"] = data["C Drug offences"] / data["Tot_Tot_P"]
new_data["saf_drug_offences"].fillna(0, inplace=True)
# Offence division D
new_data["saf_order_security"] = data["D Public order and security offences"] / data["Tot_Tot_P"]
new_data["saf_order_security"].fillna(0, inplace=True)
# Offence division E
new_data["saf_justice_procedure"] = data["E Justice procedures offences"] / data["Tot_Tot_P"]
new_data["saf_justice_procedure"].fillna(0, inplace=True)
# Offence division F
new_data["saf_other"] = data["F Other offences"] / data["Tot_Tot_P"]
new_data["saf_other"].fillna(0, inplace=True)

### Commute attributes

The code used to determine these features are located in the [1.1-CommuteAttributes.ipynb](1.1-CommuteAttributes.ipynb) jupyter notebook.

In [21]:
# read dataframe creating in other juypter notebook
suburb_campus_dist_df = pd.read_csv(os.path.join(DERIVED_DATA_PATH, "SuburbCampusDist.csv"))
# suburb_campus_dist_df.drop("id", axis=1, inplace=True)

# merge dataframes
new_data = new_data.merge(suburb_campus_dist_df, on="locality")


### Clean and export

In [22]:
export_df = new_data.drop(["lgaregion", "sa1_code_2016", "sa1_code_2021", "sa2_code_2016", "sa2_name_2016", "sa2_code_2021", "sa2_name_2021", "sa3_code_2016", "sa3_name_2016", "sa3_code_2021", "sa3_name_2021",
                          "sa4_code_2016", "sa4_name_2016", "sa4_code_2021", "sa4_name_2021", "ASGS_Structure", "Census_Code_2021", "AGSS_Code_2021", "Census_Name_2021", "Area sqkm", "lgaregion_census", ], axis=1)
export_df.replace(np.inf, 0, inplace=True)

In [23]:
export_df.to_csv("../data/derived/SuburbData.csv", index=False)